In [19]:
import os
import pandas as pd
import pptx, docx
from multiprocessing import Pool
import tqdm

#Importando programação criada pelo Thiago.
import files_indexing

In [20]:
def acessar_arquivos(Tema = 'All'):
    
    """Lista todos os caminhos possíveis da pasta raiz até todas as subpastas, acessa todos esses caminhos 
       e recupera as informações que estão em ppt ou word.
       
    param:
     Tema: Caso precise ser pego só um tema de trabalho, fazer com que Tema seja igual ao nome da pasta do tema relacionado
    
    """
    
    # Objetos que iram receber as informações finais 
    dados_finais = {}
    arquivos_error = {}
    
    if Tema == "All":
        
        #O for a seguir será um conunto de listas, cada lista possui todos os caminhos de um determinado tema
        lista_path = [[x[0] for x in os.walk(i)] for i in os.listdir() if ('.' not in i[0] and '.ipynb' not in i and '\uf00d' not in i)]
        
        #Primeiro for é para passar na lista de temas e o segundo em todos caminhos de cada tema
        for tema in tqdm.tqdm(lista_path, total = len(lista_path)):
            for path in tqdm.tqdm(tema, total = len(tema)):
                
                dados_finais.update(arquivos_da_pasta(path)[0])
                arquivos_error.update(arquivos_da_pasta(path)[1])
        
    else:
        
        #Possui um for a menos, porque já começa do Tema
        lista_path = [x[0] for x in os.walk(Tema)]
        
        for path in tqdm.tqdm(lista_path, total = len(lista_path)):
                dados_finais.update(arquivos_da_pasta(path)[0])
                arquivos_error.update(arquivos_da_pasta(path)[1])
                                            
    return dados_finais, arquivos_error

In [21]:
def arquivos_da_pasta(path):
    
    """Dado um caminho ele recupera todas as informações no formato de ppt daquele caminho
    
    param:
     path: Caminho de onde estão os arquivos que você quer recuperar
    """
    
    #Objetos para receberem valores
    dados_path = {}
    error = {}
    
    #Pegando o nome de todos os arquivos, vendo quais são maiores que 10kb e os que tem extensão doc, docx, ppt, pptx
    aux_arquivos = os.listdir(path)
    aux_arquivos = [arquivo for arquivo in aux_arquivos if os.stat(os.path.join(path,arquivo)).st_size > 10240]
    aux_arquivos = [arquivo for arquivo in aux_arquivos if '.ppt' in arquivo or '.doc' in arquivo]
    
    
    for documento in aux_arquivos:
        
        if '.ppt' in documento:
            
            #Irá testar a função criado pelo Thiago, caso der erro essa função salva o nome do arquivo e o caminho
            try: 
                texto = files_indexing.get_data_pptx(os.path.join(path, documento))
                dados_path.update({documento: texto})
            except:
                error.update({documento: path})
                
        else:
            
            #Irá testar a função criado pelo Thiago, caso der erro essa função salva o nome do arquivo e o caminho
            try: 
                texto = files_indexing.get_data_docx(os.path.join(path, documento))
                dados_path.update({documento: texto})
            except:
                error.update({documento: path})
            
            
    return dados_path, error

In [6]:
teste_dados, teste_error = acessar_arquivos('Shopping Center')

100%|██████████| 950/950 [04:27<00:00,  3.55it/s] 


In [13]:
len(teste_dados.keys())

527

In [11]:
len(teste_corrompidos.keys())

45

In [12]:
teste = ['aaa.pptx', 'sdasd', 'teste.doc', 'vai']

In [15]:
aux_arquivos = [arquivo for arquivo in teste if '.ppt' in arquivo or '.doc' in arquivo]